# RAGAS 테스트 02 - LangGraph RAG 평가

이 노트북은 LangGraph 기반 RAG 시스템의 성능을 RAGAS로 평가합니다.


In [ ]:
import requests
import json
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from datasets import Dataset

print("RAGAS 테스트 02 - LangGraph RAG 평가 시작")


In [ ]:
# LangGraph API 테스트
def test_langgraph_api(question, session_id=None):
    """LangGraph API를 호출하여 응답을 받는 함수"""
    url = "http://localhost:8001/api/v1/langgraph/multiturn"
    payload = {
        "prompt": question,
        "session_id": session_id
    }
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            return {
                "answer": data.get("response", ""),
                "sources": data.get("sources", []),
                "category": data.get("category", ""),
                "product_name": data.get("product_name", "")
            }
        else:
            return {"error": f"API 호출 실패: {response.status_code}"}
    except Exception as e:
        return {"error": f"API 호출 중 오류: {str(e)}"}

# 테스트 질문들
test_questions = [
    "KB 동반성장협약 상생대출에 대해 알려주세요",
    "KB 스마트론 대출 조건은 무엇인가요?",
    "KB 닥터론 신청 방법을 알려주세요",
    "KB 햇살론은 어떤 상품인가요?",
    "KB 전세대출 금리는 어떻게 되나요?"
]

print(f"테스트 질문 {len(test_questions)}개 준비 완료")


In [ ]:
# LangGraph API로 응답 생성
results = []
for i, question in enumerate(test_questions):
    print(f"질문 {i+1}/{len(test_questions)} 처리 중...")
    result = test_langgraph_api(question)
    results.append({
        "question": question,
        "answer": result.get("answer", ""),
        "sources": result.get("sources", []),
        "category": result.get("category", ""),
        "product_name": result.get("product_name", "")
    })

print("API 테스트 완료")


In [ ]:
# RAGAS 평가를 위한 데이터 준비
ragas_data = {
    "question": [r["question"] for r in results],
    "answer": [r["answer"] for r in results],
    "contexts": [[source.get("text", "") for source in r["sources"]] for r in results],
    "ground_truths": [
        ["KB 동반성장협약 상생대출은 중소기업 지원 대출"],
        ["KB 스마트론은 온라인 간편 대출"],
        ["KB 닥터론은 의료진 전용 대출"],
        ["KB 햇살론은 희망복지 대출"],
        ["KB 전세대출은 전세보증금 대출"]
    ]
}

dataset = Dataset.from_dict(ragas_data)
print(f"RAGAS 평가 데이터셋 생성: {len(dataset)}개 샘플")


In [ ]:
# RAGAS 평가 실행
print("RAGAS 평가 시작...")
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
)

print("\\n=== LangGraph RAG 평가 결과 ===")
print(result)
